In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras import layers, models, optimizers
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv("ecg.csv")

In [ ]:
X, y = data.iloc[:, :-1], data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

print("Before SMOTE:", np.bincount(y_train))
print("After SMOTE :", np.bincount(y_train_bal))

sns.countplot(x=y)
plt.title("Class Distribution (0 = Normal, 1 = Anomaly)")
plt.show()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_bal)
X_test_scaled  = scaler.transform(X_test)

print("Training dataset shape:", X_train_scaled.shape)
print("Testing dataset shape: ", X_test_scaled.shape)


In [ ]:

input_dim = X_train_scaled.shape[1]
print("Input dimension:", input_dim)


In [ ]:
autoencoder = models.Sequential([
    layers.Input(shape=(input_dim )),

    # Encoder
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(16, activation="tanh"),

    # Decoder
    layers.Dense(64, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(input_dim, activation="sigmoid")  
])


In [ ]:
autoencoder.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

autoencoder.summary()


In [ ]:
history = autoencoder.fit(
    X_train_scaled, X_train_scaled,
    validation_data=(X_test_scaled, X_test_scaled),
    epochs=25, batch_size=256, verbose=1
)

plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Val")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.show()

In [ ]:
# Predictions
X_train_pred = autoencoder.predict(X_train_scaled)
X_test_pred  = autoencoder.predict(X_test_scaled)

# Reconstruction error (MSE per sample)
train_errors = np.mean((X_train_scaled - X_train_pred) ** 2, axis=1)
test_errors  = np.mean((X_test_scaled - X_test_pred) ** 2, axis=1)

# Anomaly threshold
threshold = train_errors.mean() + train_errors.std()
print("Reconstruction Threshold:", threshold)


In [ ]:
# Predict anomalies
y_pred = (test_errors > threshold).astype(int)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

# Precision
precision = precision_score(y_test, y_pred)
print("Precision: ", precision)

# Recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# F1 Score
f1 = f1_score(y_test, y_pred)
print("F1 Score: ", f1)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()